<a href="https://colab.research.google.com/github/LuisDi98/Audio_a_Lesco/blob/colab/Audio_To_LESCO_models_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
#Install dependencies
! pip install datasets
! pip install transformers
! pip install -U openai-whisper
! pip install git+https://github.com/openai/whisper.git
! pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=73221e3b3324ff9434aa227723d3d0d843786379f3be7f23cf1fd49f7e969aa1
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fo

# Get Dataset
CIEMPIESS_TEST

In [33]:
#Load the dataset
from datasets import load_dataset, load_metric, Audio
import whisper
ds=load_dataset("ciempiess/ciempiess_test", split="test")

#Downsample to 16kHz
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
ds

Dataset({
    features: ['audio_id', 'audio', 'speaker_id', 'gender', 'duration', 'normalized_text'],
    num_rows: 3558
})

In [ ]:
ds[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/024bce9383676d0a3a6afc0dd8423b344044040363dd48ff5402220707a9d22c/test/male/M_07/CMPT_M_07_0074.flac',
 'array': array([-0.00192261, -0.00234985, -0.00158691, ..., -0.00839233,
        -0.00900269, -0.00698853]),
 'sampling_rate': 16000}

In [ ]:
ds.num_rows

3558

#Whisper-large
* De momento no ocupamos los inputs, esta es una manera más sencilla.
* Queda pendiente probar wav2vec2-large-xlsr-53-spanish-ep5-944h: https://huggingface.co/carlosdanielhernandezmena/wav2vec2-large-xlsr-53-spanish-ep5-944h
* Comparar con el word error rate


In [ ]:
large_model = whisper.load_model("large")
# Transcribe from audio to text with the first sample
sample = ds[0]["audio"]["path"]
result = large_model.transcribe(sample)
print("Whisper result: ",result["text"])
print("Normalized_text of the sample: ",ds[0]["normalized_text"])

100%|██████████████████████████████████████| 2.87G/2.87G [00:26<00:00, 118MiB/s]


Whisper result:   Pues ahí está la libertad de las posiciones, de a ver quién es pasivo, quién es activo, bla, bla, bla. Muchas cosas, ¿no? Pero...
Normalized_text of the sample:  pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


#Whisper-medium

In [ ]:
medium_model = whisper.load_model("medium")
# Transcribe from audio to text with the first sample
sample = ds[0]["audio"]["path"]
result = medium_model.transcribe(sample)
print("Whisper result: ",result["text"])
print("Normalized_text of the sample: ",ds[0]["normalized_text"])

100%|█████████████████████████████████████| 1.42G/1.42G [00:16<00:00, 94.3MiB/s]


Whisper result:   Pues ahí está la libertad de las posiciones, de a ver quién es pasivo, quién es activo, bla bla bla. Muchas cosas, ¿no? Pero...
Normalized_text of the sample:  pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


In [ ]:
#PRUEBAS DE LUIS DIEGO
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-medium")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="spanish", task="transcribe")


# load dummy dataset and read audio files
ds=load_dataset("ciempiess/ciempiess_test", split="test")
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
sample = ds[0]["audio"]
input_features = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").input_features
print(input_features)
# generate token ids
predicted_ids = model.generate(input_features)
print(predicted_ids)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)
print(transcription)

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print(transcription)


#Whisper-tiny

In [43]:
tiny_model = whisper.load_model("tiny")
# Transcribe from audio to text with the first sample
sample = ds[0]["audio"]["path"]
result = tiny_model.transcribe(sample)
print("Whisper result: ",result["text"] )
print("Normalized_text of the sample: ",ds[0]["normalized_text"])

100%|██████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 107MiB/s]


Whisper result:   pues esta la libertad de las posiciones de haber quien es pasivo, que en el activo la muchas cosas, ¿no? pero...
Normalized_text of the sample:  pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


#Whisper Small

In [34]:
small_model = whisper.load_model("small")
# Transcribe from audio to text with the first sample
sample = ds[0]["audio"]["path"]
result = small_model.transcribe(sample)
print("Whisper result: ",result["text"])
print("Normalized_text of the sample: ",ds[0]["normalized_text"])

Whisper result:   pues ahí está la libertad de las posiciones de haber quién es pasivo, quién es activo, bla bla bla, muchas cosas ¿no? Pero...
Normalized_text of the sample:  pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


# Normalización del texto

Necesitamos normalizar las salidas de whisper, ya que la librería nos proporciona las etiquetas de los audios pero estos están normalizados, este es un paso previo para poder aplicar algunas métricas y que los resultados sean más acertados.

In [47]:
import re
from unicodedata import normalize

def formatText(entryText):
    '''
    Quita las etiquetas xml para extraer el texto, quita caracteres,convierte a minuscula
    y devuelve lista de palabras sin acentos (Como que hace más de una cosa xd)
    '''
    text = re.sub(r'<[^<]+>', "",entryText)
    text = text.lower()
    characters = "#&¿?¡!'-[]()\/''=`,:~}{+|." #-> caracteres innecesarios que se quitan
    text = ''.join(x for x in text if x not in characters)
    return text

def deleteAccents(text):
    '''
    Esta funcion no es necesaria para Whisper
    Borrar acentos del texto, exceptuando ñ
    #Obtenido de https://ideone.com/YcXaQD (ideone.com)
    '''
    #NFD (Normalization Form Canonical Decomposition) expresion para eliminar diacríticos
    text = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", text), 0, re.I)
    #NFC (#Normalization Form Canonical Composition) volver a su forma compuesta
    text = normalize( 'NFC', text)
    return text

def deleteSpaces(entryText):
    return " ".join(entryText.split())

def normalizeText(entryText):
    newText = formatText(entryText)
    return deleteSpaces(newText)

def normalizeTextDiacritics(entryText):
    newText = formatText(entryText)
    return deleteSpaces(deleteAccents(newText))

In [48]:
normalizeText("No sé, honestamente hice hace mu#cho este código, cuando estaba en RIT & XD! [algo aqui] +, PERO me parece ineficiente")

'no sé honestamente hice hace mucho este código cuando estaba en rit xd algo aqui pero me parece ineficiente'

#WORD ERROR RATE
Se tiene una referencia, en este caso el label del audio y se tiene el texto generado por whisper.

WER = (S+D+I) / N => WER = (S+D+I) / (S+D+C)
​

Dónde:
* S es el número de sustituciones,
* D es el número de eliminaciones,
* I es el número de inserciones,
* C es el número de aciertos,
* N es el número de palabras en la referencia, N = (S + D + C).

A veces tambien se usa el Word accuracy, Auto Speech Recognition.

Wacc = 1 - WER

In [24]:
! pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.8 MB/s eta 0:00:00


In [35]:
import jiwer

def getWER(ref, hyp):
  '''
  ref is the ground truth, original
  hyp is the generated text to compare with the original
  '''
  error = jiwer.wer(ref, hyp)
  return error

In [36]:
# El modelo que quieran
# tiny_model, medium_model, large_model
result = small_model.transcribe(sample)

In [39]:
print("WER para el primer simple:", getWER((result["text"]), ds[0]["normalized_text"]))
print((result["text"]),"\n", ds[0]["normalized_text"])

print("WER para el primer simple:", getWER(ds[0]["normalized_text"], (result["text"])))
print((result["text"]),"\n", ds[0]["normalized_text"])

WER para el primer simple: 0.4782608695652174
 pues ahí está la libertad de las posiciones de haber quién es pasivo, quién es activo, bla bla bla, muchas cosas ¿no? Pero... 
 pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero
WER para el primer simple: 0.5238095238095238
 pues ahí está la libertad de las posiciones de haber quién es pasivo, quién es activo, bla bla bla, muchas cosas ¿no? Pero... 
 pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


In [ ]:
print("WER para el primer texto normalizado:", getWER(normalizeText(result["text"]), ds[0]["normalized_text"]))
print(normalizeText(result["text"]),"\n", ds[0]["normalized_text"])

WER para el primer texto normalizado: 0.30434782608695654
pues ahí está la libertad de las posiciones de haber quién es pasivo quién es activo bla bla bla muchas cosas no pero 
 pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


In [ ]:
print("WER para el primer normalizado sin diacriticos:", getWER(normalizeTextDiacritics(result["text"]), ds[0]["normalized_text"]))
print(normalizeTextDiacritics(result["text"]),"\n", ds[0]["normalized_text"])

WER para el primer normalizado sin diacriticos: 0.43478260869565216
pues ahi esta la libertad de las posiciones de haber quien es pasivo quien es activo bla bla bla muchas cosas no pero 
 pues está la libertá de las posiciones de a ver quién es pasivo quién es activo blablablá muchas cosas no pero


#WER para todo el data set de Ciempies con Whisper tiny
* Tiempo aproximado con 1000 audios:
  * Tiny: 7,5 mins
  * Small: 16,7 mins
  * Medium: 32,8 mins
  * Large: 46,5 mins



In [40]:
import numpy as np

def getWERMetric(model, dataset, datasetSample):
  WERList = []
  for i in range(datasetSample):
    sample = dataset[i]["audio"]["path"]
    result = model.transcribe(sample)
    textResult = normalizeText(result["text"])
    label = dataset[i]["normalized_text"]
    if (textResult!="" and label!=""):
      WERList.append(getWER(label, textResult))
  return np.mean(WERList)

In [41]:
import time

In [ ]:
#Esto es generado vs original, pero lo correcto es el de abajo
start = time.time()
value = getWERMetric(tiny_model,ds,1000)
end = time.time()
print(value)
print(end-start)

0.2917494913844536
455.01868629455566


In [49]:
#Esto es original vs generado
start = time.time()
value = getWERMetric(tiny_model,ds,1000)
end = time.time()
print(value)
print(end-start)

0.25775208194565835
584.2194185256958


In [ ]:
start = time.time()
value = getWERMetric(small_model,ds,1000)
end = time.time()
print(value)
print(end-start)

0.18433433443007663
1003.1085572242737


In [ ]:
start = time.time()
value = getWERMetric(medium_model,ds,1000)
end = time.time()
print(value)
print(end-start)

0.17623326276082082
1970.6431975364685


In [ ]:
start = time.time()
value = getWERMetric(large_model,ds,1000)
end = time.time()
print(value)
print(end-start)

0.19101982265768988
2790.7779171466827


# NEW Voice Activity Detector


In [ ]:
#@title Install and Import Dependencies
# this assumes that you have a relevant version of PyTorch installed
!pip install -q torchaudio
import torch
torch.set_num_threads(1)
from IPython.display import Audio
from pprint import pprint
USE_ONNX = False # change this to True if you want to test onnx model
if USE_ONNX:
  !pip install -q onnxruntime

In [ ]:
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True,
                              onnx=USE_ONNX)
(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip


In [ ]:
def chunk_audio(audio):
  readAudio = read_audio(audio, sampling_rate=16000)
  # get speech timestamps from full audio file
  speech_timestamps = get_speech_timestamps(readAudio, model, sampling_rate=16000)
  return (speech_timestamps,readAudio)

def getAudioVAD(audio):
   audio_chunks = chunk_audio(audio)
   return collect_chunks(audio_chunks[0], audio_chunks[1])

# merge all speech chunks to one audio
# audio_chunks = chunk_audio('/content/Prueba.m4a')
# save_audio('new_audio_VAD.wav', collect_chunks(audio_chunks[0], audio_chunks[1]), sampling_rate=16000)
# Audio('new_audio_VAD.wav')
# In this example the original audio was 17 sec, the new one is 10 s and VAD only took a couple of seconds

#Wav2VEC2


In [ ]:
import soundfile as sf
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [ ]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-spanish")

# Set the desired tokenization parameters
max_length = 100  # Maximum length of the input in number of tokens
padding="True" # Whether to pad the sequences to have the same length
padding_side = "right"  # Side to apply padding on ("right" or "left")
truncation="True" # Whether to truncate the sequences if they exceed the maximum length
stride = 0.2  # Stride length for splitting audio into overlapping windows

# Update the tokenizer's parameters
tokenizer.model_max_length = max_length
tokenizer.padding = padding
tokenizer.padding_side = padding_side
tokenizer.truncation = truncation
tokenizer.stride = stride

model = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-spanish")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.


In [ ]:
def transcribe_audio(audio):
    input_values = tokenizer(audio, return_tensors="pt").input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    return transcription


In [ ]:
from jiwer import wer


In [ ]:
def conseguirWER(model,dataset,datasetSample):
  total_wer = 0.0
  total_audio_files = datasetSample

  for i in range(datasetSample):
      audio_path = dataset[i]["audio"]["path"]
      ground_truth_transcription = dataset[i]["normalized_text"]


      audio_dict = dataset[i]["audio"]
      audio, sample_rate = sf.read(audio_dict["path"])
      # Transcribe audio using the model
      transcription = transcribe_audio(audio)

      # Calculate Word Error Rate (WER)
      wer_score = wer(ground_truth_transcription, transcription)
      total_wer += wer_score

      average_wer = total_wer / total_audio_files
      print("Average WER:", average_wer)
      return average_wer


In [ ]:
start = time.time()
value = conseguirWER(model,ds,2000)
end = time.time()
print(value)
print(end-start)

Average WER: 0.0002380952380952381
0.0002380952380952381
11.46746039390564


In [ ]:
ds[0]["audio"]["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/024bce9383676d0a3a6afc0dd8423b344044040363dd48ff5402220707a9d22c/test/male/M_07/CMPT_M_07_0074.flac'

In [ ]:
ds[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/024bce9383676d0a3a6afc0dd8423b344044040363dd48ff5402220707a9d22c/test/male/M_07/CMPT_M_07_0074.flac',
 'array': array([-0.00192261, -0.00234985, -0.00158691, ..., -0.00839233,
        -0.00900269, -0.00698853]),
 'sampling_rate': 16000}

In [ ]:
audio_dict = ds[0]["audio"]

In [ ]:
audio_dict["path"]

'/root/.cache/huggingface/datasets/downloads/extracted/024bce9383676d0a3a6afc0dd8423b344044040363dd48ff5402220707a9d22c/test/male/M_07/CMPT_M_07_0074.flac'

In [ ]:
import soundfile as sf

In [ ]:
audio, sample_rate = sf.read(audio_dict["path"])

# Test with large audios
Estas pruebas son usando los diferentes modelos de whisper y sin Voice Activity Detector.
1. Primero se debe correr el modelo de whisper que se desea.
2. Luego la funcion para transcirbir el audio, esta a su vez da el tiempo que se tarda.

Pruebas para el modelo Small
* Audio 30 mins           = 4.81 mins
* Audio 1 hora            = 5.99 mins
* Audio 1 hora y 30 mins  = 5.62 mins
* Audio 2 horas           = 6.68 mins
* Audio 2 horas y 30 mins = 15.1 mins
* Audio 3 horas           = 7.41 mins


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import whisper
import time
#small_model = whisper.load_model("small")
#tiny_model = whisper.load_model("tiny")
#medium_model = whisper.load_model("medium")
large_model = whisper.load_model("large")

Mounted at /content/drive


100%|██████████████████████████████████████| 2.88G/2.88G [00:29<00:00, 103MiB/s]


In [ ]:
def transcribeAudio(audio, model):
  start = time.time()
  result = model.transcribe(audio)
  end = time.time()
  return (end-start, result)

In [ ]:
smallTime = {"30":0, "60":0, "90":0, "120":0, "150":0, "180":0}
audio1= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/30Min.mp3',small_model)
smallTime["30"] = audio1[0]
audio2= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/60Min.mp3',small_model)
smallTime["60"] = audio2[0]
audio3= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/90Min.mp3',small_model)
smallTime["90"] = audio3[0]
audio4= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/120Min.mp3',small_model)
smallTime["120"] = audio4[0]
audio5= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/150Min.mp3',small_model)
smallTime["150"] = audio5[0]
audio6= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/180Min.mp3',small_model)
smallTime["180"] = audio6[0]

In [ ]:
smallTime
'''
{'30': 289.04940462112427,
 '60': 359.7411675453186,
 '90': 337.9016089439392,
 '120': 400.85388445854187,
 '150': 906.0902349948883,
 '180': 1094.6138291358948
}
'''

Pruebas para el modelo Medium
* Audio 30 mins           =  5.94 mins
* Audio 1 hora            =  9.72 mins
* Audio 1 hora y 30 mins  =  15.91 mins
* Audio 2 horas           =  23.92 mins
* Audio 2 horas y 30 mins =  21.98 mins
* Audio 3 horas           =  20.80 mins

In [ ]:
mediumTime = {"30":0, "60":0, "90":0, "120":0, "150":0, "180":0}
audio1= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/30Min.mp3', medium_model)
mediumTime["30"] = audio1[0]
audio2= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/60Min.mp3', medium_model)
mediumTime["60"] = audio2[0]
audio3= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/90Min.mp3', medium_model)
mediumTime["90"] = audio3[0]
audio4= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/120Min.mp3', medium_model)
mediumTime["120"] = audio4[0]
audio5= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/150Min.mp3', medium_model)
mediumTime["150"] = audio5[0]
audio6= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/180Min.mp3', medium_model)
mediumTime["180"] = audio6[0]
#1 h,38 mins

In [ ]:
mediumTime

{'30': 356.7253592014313,
 '60': 583.5891742706299,
 '90': 954.7192177772522,
 '120': 1435.6262798309326,
 '150': 1319.2298076152802,
 '180': 1248.4831037521362}

# Test with VAD + Dataset

Modelo Small

In [ ]:
smallTimeVAD = {"30":0, "60":0, "90":0, "120":0, "150":0, "180":0}
audio1= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/30Min.mp3'), small_model)
smallTimeVAD["30"] = audio1[0]
audio2= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/60Min.mp3'), small_model)
smallTimeVAD["60"] = audio2[0]
audio3= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/90Min.mp3'), small_model)
smallTimeVAD["90"] = audio3[0]
audio4= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/120Min.mp3'), small_model)
smallTimeVAD["120"] = audio4[0]
audio5= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/150Min.mp3'), small_model)
smallTimeVAD["150"] = audio5[0]
audio6= transcribeAudio(getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/180Min.mp3'), small_model)
smallTimeVAD["180"] = audio6[0]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501: UserWarning: operator() profile_node %669 : int[] = prim::profile_ivalue(%667)
 does not have profile information (Triggered internally at ../third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  return forward_call(*args, **kwargs)


In [ ]:
smallTimeVAD
#ESTOS TIEMPOS PARECEN MEJORES PERO NO INCLUYEN EL TIEMPO DEL VAD

{'30': 174.00128769874573,
 '60': 314.86626648902893,
 '90': 396.40985894203186,
 '120': 550.9504287242889,
 '150': 658.8372821807861,
 '180': 775.54731798172}

Modelo medium

In [ ]:
mediumTimeVAD = {"30":0, "60":0, "90":0, "120":0, "150":0, "180":0}

In [ ]:

start = time.time()
a1= getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/30Min.mp3')
end = time.time()
print(end-start)

start = time.time()
a2= getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/60Min.mp3')
end = time.time()
print(end-start)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501: UserWarning: operator() profile_node %669 : int[] = prim::profile_ivalue(%667)
 does not have profile information (Triggered internally at ../third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  return forward_call(*args, **kwargs)


117.35872101783752
209.56743001937866


"\na3= getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/90Min.mp3')\na4= getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/120Min.mp3')\na5= getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/150Min.mp3')\na6= getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/180Min.mp3')\n"

In [ ]:
start = time.time()
a3= getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/90Min.mp3')
end = time.time()
print(end-start)

341.01606917381287


In [ ]:
start = time.time()
a4= getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/120Min.mp3')
end = time.time()
print(end-start)

412.2972481250763


In [ ]:
start = time.time()
a5= getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/150Min.mp3')
end = time.time()
print(end-start)

509.9150776863098


In [ ]:
start = time.time()
a6= getAudioVAD('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/180Min.mp3')
end = time.time()
print(end-start)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501: UserWarning: operator() profile_node %669 : int[] = prim::profile_ivalue(%667)
 does not have profile information (Triggered internally at ../third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  return forward_call(*args, **kwargs)


632.9283192157745


In [ ]:
audio1= transcribeAudio(a1, medium_model)
mediumTimeVAD["30"] = audio1[0]

In [ ]:
audio2= transcribeAudio(a2, medium_model)
mediumTimeVAD["60"] = audio2[0]

In [ ]:
audio3= transcribeAudio(a3, medium_model)
mediumTimeVAD["90"] = audio3[0]

In [ ]:
audio4= transcribeAudio(a4, medium_model)
mediumTimeVAD["120"] = audio4[0]

In [ ]:
audio5= transcribeAudio(a5, medium_model)
mediumTimeVAD["150"] = audio5[0]

In [ ]:
audio6= transcribeAudio(a6, medium_model)
mediumTimeVAD["180"] = audio6[0]

In [ ]:
mediumTimeVAD

{'30': 0, '60': 0, '90': 0, '120': 0, '150': 0, '180': 1777.4128229618073}

# Generate new dataset (Audios del consejo estudiantil)

In [ ]:
! pip install audiosegment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp310-cp310-linux_x86_64.whl size=73473 sha256=77bb854f202d1a1a0685ea77c106de3b868a1e4671037defddc7b97ccd1237a0
  Stored in directory: /root/.cache/pip/wheels/2a/2b/84/ac7bacfe8c68a87c1ee3dd3c66818a54c71599abf308e8eb35
Successfully built webrtcvad


In [ ]:
from pydub import AudioSegment

def audiolenght(input_file):
    audio = AudioSegment.from_file(input_file)
    return len(audio)

def cutAudio(input_file):
    audiolen = audiolenght(input_file)
    print(audiolen)
    audio = AudioSegment.from_file(input_file)
    counter = 0
    start_time = 0
    end_time = 300000
    while end_time < audiolen:
      audio_cut = audio[start_time:end_time]
      start_time = end_time
      end_time+= 300000
      print("end_time",end_time)
      audio_cut.export("/content/audio"+str(counter)+".mp3", format="mp3")
      counter+= 1

# Ejemplo de uso
input_file = "/content/Consejo.mp3"
cutAudio(input_file)


8921040
end_time 600000
end_time 900000
end_time 1200000
end_time 1500000
end_time 1800000
end_time 2100000
end_time 2400000
end_time 2700000
end_time 3000000
end_time 3300000
end_time 3600000
end_time 3900000
end_time 4200000
end_time 4500000
end_time 4800000
end_time 5100000
end_time 5400000
end_time 5700000
end_time 6000000
end_time 6300000
end_time 6600000
end_time 6900000
end_time 7200000
end_time 7500000
end_time 7800000
end_time 8100000
end_time 8400000
end_time 8700000
end_time 9000000


In [ ]:
import zipfile

# output zip filename
zip_filename = 'datasetConsejo.zip'

# Create zip and add files
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for i in range(0, 29):
      zipf.write("/content/audio"+str(i)+".mp3")
zipf.close()

# Transcribir audios (Dataset 5 mins)

In [ ]:
transcripts = {}
for i in range (15, 30):
  print(i)
  audio= transcribeAudio('/content/drive/MyDrive/DataSet-ConsejoEstudiantil/Dataset-5mins/audio'+str(i)+'.mp3',medium_model)
  transcripts[str(i)] = audio

15
16
17
18
19
20
21
22
23
24
25
26
27
28


In [ ]:
print(transcripts['15'][1]["text"])

 más o menos la mitad de todos los graduados en áreas STEM son mujeres. Sin embargo, esto tiene diferencias muy importantes cuando uno desagrega estas áreas por disciplinas. Donde las mujeres han progresado mucho en las áreas de ciencias médicas o en ciencias de la salud, fundamentalmente tenemos una enorme cantidad de mujeres que andan graduándose de enfermería. Y entonces esto altera montones los promedios de los indicadores. Cuando uno controla esto y mira las otras áreas, la verdad es que hay problemas importantes, como mencionaba, en las áreas de ingeniería y tecnología. El gráfico que mostré en el capítulo anterior de cómo ha venido cayendo el porcentaje de personas que logra obtener su primer título, el indicador de eficiencia terminal de las cortes, cuando uno lo parte entre áreas STEM y no STEM y entre hombres y mujeres, encuentra un indicador muy preocupante, que es que está cayendo, de todas las caídas... La más crítica es de mujeres en áreas STEM, donde más se está contraye

#WER con dataset consejo estudiantil Youtube (5 mins)

In [1]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 2.7 MB/s eta 0:00:00


#### Download the datasets Transcripts and Corrections to get the WER

In [82]:
import os
from docx import Document
import numpy as np

#Create these folders and upload the files locally to colab
directory_path_original = '/content/Original/'
directory_path_corrected = '/content/Corrected/'
WERList = []

for file_number in range(15,30):
  # List all file in the folder
  files_original = os.listdir(directory_path_original)
  files_corrected = os.listdir(directory_path_corrected)

  # Search the file
  file_path_original = [filename for filename in files_original if f"{file_number}" in filename]
  file_path_corrected = [filename for filename in files_corrected if f"{file_number}" in filename]

  # Create the document instance
  doc_original = Document(directory_path_original + file_path_original[0])
  doc_corrected = Document(directory_path_corrected + file_path_corrected[0])

  content_original = []
  content_corrected = []

  for paragraph in doc_original.paragraphs:
      content_original.append(paragraph.text)

  for paragraph in doc_corrected.paragraphs:
      content_corrected.append(paragraph.text)

  # File original or correct that we corrected to be the reference vs generated by Whisper which is the original with no changes
  WERList.append(getWER(content_corrected[0], content_original[0]))

print(WERList)
print("Mean WER", np.mean(WERList))

[0.12157330154946365, 0.13324360699865412, 0.011461318051575931, 0.0, 0.02531645569620253, 0.023255813953488372, 0.007707129094412331, 0.019332161687170474, 0.023006134969325152, 0.014947683109118086, 0.058361391694725026, 0.008168028004667444, 0.03916768665850673, 0.0030165912518853697, 0.03134328358208955]
Mean WER 0.03466003908675232
